In [67]:
from shapely.geometry import box

In [64]:
def get_raster_wkt(raster):
    
    src = gdal.Open(raster)
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)

    from osgeo import ogr
    from osgeo import osr

    # Setup the source projection - you can also import from epsg, proj4...
    source = osr.SpatialReference()
    source.ImportFromWkt(src.GetProjection())

    # The target projection
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)

    # Create the transform - this can be used repeatedly
    transform = osr.CoordinateTransformation(source, target)

    return box(transform.TransformPoint(ulx, lry)[0], 
       transform.TransformPoint(ulx, lry)[1],
       transform.TransformPoint(lrx, uly)[0],
       transform.TransformPoint(lrx, uly)[1]).wkt


In [147]:
base_url = 'hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/'
band_to_process = 'B01'
references = []

for tile in ['0_0', '0_1']:
    
    for prefix in [band_to_process, 'original_{}'.format(band_to_process), 's_{}'.format(band_to_process)]:
        
        references.append('{}{}_tile_{}_T36PXQ.pickle.tif'.format(base_url, prefix, tile))

In [148]:
references

['hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/B01_tile_0_0_T36PXQ.pickle.tif',
 'hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/original_B01_tile_0_0_T36PXQ.pickle.tif',
 'hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/s_B01_tile_0_0_T36PXQ.pickle.tif',
 'hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/B01_tile_0_1_T36PXQ.pickle.tif',
 'hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/original_B01_tile_0_1_T36PXQ.pickle.tif',
 'hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/s_B01_tile_0_1_T36PXQ.pickle.tif']

In [153]:
os.path.basename(references[0]).split('_')[-1].split('.')[0]

'T36PXQ'

In [133]:
import pandas as pd
import cioppy 
import os
import shutil
import gdal
import lxml.etree as etree
import numpy as np
from shapely.wkt import loads


In [154]:
df_references = pd.DataFrame(references)

In [155]:
df_references.columns = ['enclosure']

In [156]:
df_references


,enclosure
0,hdfs://sb-10-150-0-21.better.terradue.int:8020...
1,hdfs://sb-10-150-0-21.better.terradue.int:8020...
2,hdfs://sb-10-150-0-21.better.terradue.int:8020...
3,hdfs://sb-10-150-0-21.better.terradue.int:8020...
4,hdfs://sb-10-150-0-21.better.terradue.int:8020...
5,hdfs://sb-10-150-0-21.better.terradue.int:8020...


In [157]:
def analyse_merge_row(row, band_to_process):
    
    series = dict()

    if 's_' in row.enclosure:
        output_type = 's'
    
    elif 'original_{}'.format(band_to_process) in row.enclosure:
        output_type = 'original_{}'.format(band_to_process)
        
    else: 
        output_type = band_to_process

    series['output_type'] = output_type
    series['tile'] = os.path.basename(row.enclosure).split('_')[-1].split('.')[0]
    return pd.Series(series)    


In [158]:
df_references = df_references.merge(df_references.apply(lambda row: analyse_merge_row(row, band_to_process), axis=1), 
              left_index=True,
              right_index=True)

In [159]:
df_references

,enclosure,output_type,tile
0,hdfs://sb-10-150-0-21.better.terradue.int:8020...,B01,T36PXQ
1,hdfs://sb-10-150-0-21.better.terradue.int:8020...,original_B01,T36PXQ
2,hdfs://sb-10-150-0-21.better.terradue.int:8020...,s,T36PXQ
3,hdfs://sb-10-150-0-21.better.terradue.int:8020...,B01,T36PXQ
4,hdfs://sb-10-150-0-21.better.terradue.int:8020...,original_B01,T36PXQ
5,hdfs://sb-10-150-0-21.better.terradue.int:8020...,s,T36PXQ


In [160]:
df_references[df_references.output_type == band_to_process]

,enclosure,output_type,tile
0,hdfs://sb-10-150-0-21.better.terradue.int:8020...,B01,T36PXQ
3,hdfs://sb-10-150-0-21.better.terradue.int:8020...,B01,T36PXQ


In [161]:
ciop = cioppy.Cioppy()

In [162]:
os.environ['HOME'] = '/home/fbrito'

In [164]:
results = {}
tile = df_references.tile.unique()[0]
for output_type in df_references.output_type.unique():
    
    print (output_type)
    local_paths = [] 

    
    for entry, row in df_references[df_references.output_type == output_type].iterrows():
   
        print row.enclosure
        local_paths.append(ciop.copy(row.enclosure, '.'))
        
    local_vrt = '{}_temp_vrt.xml'.format(output_type)

    ds = gdal.BuildVRT(local_vrt, local_paths)

    ds.FlushCache()
    
    src_ds = gdal.Open(local_paths[0])
    
    # save individual bands
    for index in range(src_ds.RasterCount):
        
        desc = src_ds.GetRasterBand(index+1).GetDescription()
        meta = src_ds.GetRasterBand(index+1).GetMetadata()
        
        if desc == '':
            #s
            output_name = '{}_{}'.format(tile, output_type)
            translated_tif = '{}.tif'.format(output_name)
        else:
            if 'band_is_interpolated' in meta.keys():
                # interpolated
                output_name = '{}_{}_{}_{}_{}'.format(tile,
                                                      band_to_process,
                                                    desc,
                                                    meta['date'],
                                                   'SYN' if meta['band_is_interpolated'] == 'True' else 'NAT')
                
                translated_tif = '{}.tif'.format(output_name)
                
                
                    
            else:
                # original
                output_name = '{}_{}_{}_{}_{}'.format(tile,
                                                   band_to_process,
                                                    desc,
                                                    meta['date'],
                                                   'ORI')
                
                translated_tif = '{}.tif'.format(output_name)
        
        gdal.Translate(translated_tif,
                       local_vrt, 
                      bandList=[index+1])
    
        ds = gdal.Open(translated_tif,  gdal.OF_UPDATE)
        ds.GetRasterBand(1).SetDescription(desc)
        ds.GetRasterBand(1).SetMetadata(meta)
        ds.FlushCache()
        if desc == '':
            
            title = 'Sentinel-2 tile {} {} s factor'.format(tile, band_to_process)
            
        else:
            prd_type = 'original' if not 'band_is_interpolated' in meta.keys() else 'synthetic' if meta['band_is_interpolated'] == 'True' else 'native'
                
            title = 'Sentinel-2 tile {} {} {} Julian day {}, date {}'.format(tile,
                                                                          band_to_process,
                                                                             prd_type,
                                                                             meta['date'], 
                                                                             datetime.datetime.strptime(desc, '%Y%j').date().strftime('%Y-%m-%d'))

            metadata = dict()

            metadata['identifier'] = '_'.join([output_name, tile])
            metadata['startdate'] = datetime.datetime.strptime(desc, '%Y%j').date().strftime('%Y-%m-%dT00:00:00Z')
            metadata['enddate'] = datetime.datetime.strptime(desc, '%Y%j').date().strftime('%Y-%m-%dT23:59:59Z')
            metadata['product_type'] = 'S2_{}_{}'.format(band_to_process, 'ORI' if not 'band_is_interpolated' in meta.keys() else 'SYN' if meta['band_is_interpolated'] == 'True' else 'NAT')
            metadata['wkt'] = get_raster_wkt(translated_tif)

            eop_xml = '{}.xml'.format(output_name)
            with open(eop_xml, 'wb') as file:
                file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
                file.write(create_metadata(metadata))

            with open('{}.properties'.format(output_name), 'wb') as file:
                file.write('title={}\n'.format(title))

        
        
    for file in (local_paths + [local_vrt]):
        
        os.remove(file)

B01
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/B01_tile_0_0_T36PXQ.pickle.tif
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/B01_tile_0_1_T36PXQ.pickle.tif
original_B01
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/original_B01_tile_0_0_T36PXQ.pickle.tif
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/original_B01_tile_0_1_T36PXQ.pickle.tif
s
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/s_B01_tile_0_0_T36PXQ.pickle.tif
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000002-191101000012551-oozie-oozi-W/subtile/data/s_B01_tile_0_1_T36PXQ.pickle.tif


In [94]:
a = dict()
a['b'] = ''
if 'b' in a.keys():
    print 'yes'

yes


In [75]:
    gdal.Translate(translated_tif,
                       local_vrt)
    
      
    src_ds = gdal.Open(local_paths[0])
    ds = gdal.Open(translated_tif,  gdal.OF_UPDATE)

    bands = []

    for index in range(src_ds.RasterCount):
       
        bands.append(src_ds.GetRasterBand(index+1).GetDescription())
        ds.GetRasterBand(index+1).SetDescription(src_ds.GetRasterBand(index+1).GetDescription())

    ds.FlushCache()
    
    for file in (local_paths + [local_vrt]):
        
        os.remove(file)
    
    if bands[0] == '':
        results[output_type] = {'file': translated_tif}
    else:                                                           
        shutil.move(translated_tif, '{}_{}_{}.tif'.format(output_type, bands[0], bands[-1]))
    
        results[output_type] = {'file': '{}_{}_{}.tif'.format(output_type, bands[0], bands[-1]),
                           'min_julian': bands[0],
                           'max_julian': bands[-1]}

B01
B01
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000000-191030000011463-oozie-oozi-W/subtile/data/B01_tile_0_0.pickle.tif
B01
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000000-191030000011463-oozie-oozi-W/subtile/data/B01_tile_0_1.pickle.tif
original_B01
original_B01
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000000-191030000011463-oozie-oozi-W/subtile/data/original_B01_tile_0_0.pickle.tif
original_B01
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000000-191030000011463-oozie-oozi-W/subtile/data/original_B01_tile_0_1.pickle.tif
s
s
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000000-191030000011463-oozie-oozi-W/subtile/data/s_B01_tile_0_0.pickle.tif
s
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000000-191030000011463-oozie-oozi-W/subtile/data/s_B01_tile_0_1.pickle.tif


In [76]:
results

{'B01': {'file': 'B01_2019045_2019150.tif',
  'max_julian': '2019150',
  'min_julian': '2019045'},
 'original_B01': {'file': 'original_B01_2019045_2019150.tif',
  'max_julian': '2019150',
  'min_julian': '2019045'},
 's': {'file': 's.tif'}}

In [97]:
def create_metadata(metadata):

    namespaces = dict()

    namespaces['opt'] = 'http://www.opengis.net/opt/2.1'
    namespaces['om']  = 'http://www.opengis.net/om/2.0'
    namespaces['gml'] = 'http://www.opengis.net/gml/3.2'
    namespaces['eop'] = 'http://www.opengis.net/eop/2.1'
    namespaces['sar'] = 'http://www.opengis.net/sar/2.1'
    namespaces['ssp'] = 'http://www.opengis.net/ssp/2.1'
    
    
    for key, value in namespaces.items():
        etree.register_namespace(key, value)
   
    root = etree.Element('{{}}EarthObservation'.format(namespaces['ssp']))

    # Time
    phenomenon_time = etree.SubElement(root, '{{{}}}phenomenonTime'.format(namespaces['om']))
    time_period = etree.SubElement(phenomenon_time, '{{{}}}TimePeriod'.format(namespaces['gml']))
    begin_position = etree.SubElement(time_period, '{{{}}}beginPosition'.format(namespaces['gml']))
    end_position = etree.SubElement(time_period, '{{{}}}endPosition'.format(namespaces['gml']))
    
    feature_of_interest = etree.SubElement(root, '{{{}}}featureOfInterest'.format(namespaces['om']))
    footprint = etree.SubElement(feature_of_interest, '{{{}}}Footprint'.format(namespaces['ssp']))
    multi_extentOf = etree.SubElement(footprint, '{{{}}}multiExtentOf'.format(namespaces['ssp']))
    multi_surface = etree.SubElement(multi_extentOf, '{{{}}}MultiSurface'.format(namespaces['gml']))
    surface_members = etree.SubElement(multi_surface, '{{{}}}surfaceMembers'.format(namespaces['gml']))
    polygon = etree.SubElement(surface_members, '{{{}}}Polygon'.format(namespaces['gml']))
    exterior = etree.SubElement(polygon, '{{{}}}exterior'.format(namespaces['gml']))
    linear_ring = etree.SubElement(exterior, '{{{}}}LinearRing'.format(namespaces['gml']))
    poslist = etree.SubElement(linear_ring, '{{{}}}posList'.format(namespaces['gml']))

    # Metadata property
    metadata_property = etree.SubElement(root, '{{{}}}metaDataProperty'.format(namespaces['eop']))
    earth_observation_metadata = etree.SubElement(metadata_property, '{{{}}}EarthObservationMetaData'.format(namespaces['eop']))
    identifier = etree.SubElement(earth_observation_metadata, '{{{}}}identifier'.format(namespaces['eop']))
    product_type = etree.SubElement(earth_observation_metadata, '{{{}}}productType'.format(namespaces['eop']))
    
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
   
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 
    product_type.text = metadata['product_type'] 
    
    return etree.tostring(root, pretty_print=True)


In [73]:
for key in results.keys():
        print key
        ciop.log('INFO', 'key: {}'.format(key))
        if key == 's': 

            title = 'Sentinel-2 {} s factor from {} to {}'.format(band_to_process, results[band_to_process]['min_julian'], results[band_to_process]['max_julian'])

            prd_type = 'S'

            s_output_name = 'S2_{}_{}_J{}_J{}.tif'.format(band_to_process, prd_type, results[band_to_process]['min_julian'], results[band_to_process]['max_julian'])

            src_ds = gdal.Open(results[key]['file'])

            drv = gdal.GetDriverByName('GTiff')

            ciop.log('INFO', 'Creating product {} type {}'.format(s_output_name, prd_type))

            ds = drv.Create(s_output_name,
                            src_ds.RasterXSize, 
                            src_ds.RasterYSize, 
                            src_ds.RasterCount, 
                            gdal.GDT_Float32)

            ds.SetGeoTransform(src_ds.GetGeoTransform())
            ds.SetProjection(src_ds.GetProjection())

            ds.GetRasterBand(1).WriteArray(src_ds.GetRasterBand(1).ReadAsArray(), 0, 0)
            ds.FlushCache()
            
            metadata = dict()
            
            metadata['identifier'] = s_output_name.replace('.tif', '')
            metadata['startdate'] = datetime.datetime.strptime(results[band_to_process]['min_julian'], '%Y%j').date().strftime('%Y-%m-%dT00:00:00Z')
            metadata['enddate'] = datetime.datetime.strptime(results[band_to_process]['max_julian'], '%Y%j').date().strftime('%Y-%m-%dT23:59:59Z')
            metadata['product_type'] = 'S2_S'
            metadata['wkt'] =get_raster_wkt(s_output_name)
            
            eop_xml = s_output_name.replace('.tif', '.xml')
            with open(eop_xml, 'wb') as file:
                file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
                file.write(create_metadata(metadata))
            
            with open(s_output_name.replace('.tif', '.properties'), 'wb') as file:
                file.write('title={}\n'.format(title))
                
            ciop.log('INFO', 'Publish {}'.format(s_output_name))
            ciop.publish(os.path.join(ciop.tmp_dir, s_output_name), metalink=True)
            ciop.publish(os.path.join(ciop.tmp_dir, s_output_name.replace('.tif', '.properties')), metalink=True)
            ciop.publish(os.path.join(ciop.tmp_dir, s_output_name.replace('.xml', '.properties')), metalink=True)
    
        else:    

            #####
            bands = []
            meta = []
            ciop.log('INFO', 'Opening {} - {}'.format(key, results[key]['file']))

            src_ds = gdal.Open(results[key]['file'])

            for index in range(src_ds.RasterCount):

                description = src_ds.GetRasterBand(index+1).GetDescription()

                meta = src_ds.GetRasterBand(index+1).GetMetadata()
                print index
                print meta
                if key == 'original_{}'.format(band_to_process):

                    title = 'Sentinel-2 Original {}'.format(band_to_process)
                    prd_type = 'ORI'

                else:

                    if meta['band_is_interpolated'] == 'True':

                        prd_type = 'SYN'
                        title = 'Sentinel-2 {} {} Julian day {}, date {}'.format(band_to_process, 'synthetic', description, datetime.datetime.strptime(description, '%Y%j').date().strftime('%Y-%m-%d'))
                    else:

                        prd_type = 'NAT'
                        title = 'Sentinel-2 {} {} Julian day {}, date {}'.format(band_to_process, 'native', description, datetime.datetime.strptime(description, '%Y%j').date().strftime('%Y-%m-%d'))

                daily_output_name = 'S2_{}_{}_J{}_{}.tif'.format(band_to_process, prd_type, description, meta['date'])

                drv = gdal.GetDriverByName('GTiff')

                ciop.log('INFO', 'Creating product {} type {}'.format(daily_output_name, prd_type))

                ds = drv.Create(daily_output_name, 
                                    src_ds.RasterXSize, 
                                src_ds.RasterYSize, 
                                    src_ds.RasterCount, 
                                    gdal.GDT_Float32)

                ds.SetGeoTransform(src_ds.GetGeoTransform())
                ds.SetProjection(src_ds.GetProjection())

                #a = src_ds.GetRasterBand(index+1)
                ds.GetRasterBand(1).WriteArray(src_ds.GetRasterBand(index+1).ReadAsArray(), 0, 0)
                ds.FlushCache()

                
                metadata = dict()
            
                metadata['identifier'] = daily_output_name.replace('.tif', '')
                metadata['startdate'] = datetime.datetime.strptime(description, '%Y%j').date().strftime('%Y-%m-%dT00:00:00Z')
                metadata['enddate'] = datetime.datetime.strptime(description, '%Y%j').date().strftime('%Y-%m-%dT23:59:59Z')
                metadata['product_type'] = 'S2_{}_{}'.format(band_to_process, prd_type)
                metadata['wkt'] = get_raster_wkt(daily_output_name)

                eop_xml = daily_output_name.replace('.tif', '.xml')
                with open(eop_xml, 'wb') as file:
                    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
                    file.write(create_metadata(metadata))
                
                with open(daily_output_name.replace('.tif', '.properties'), 'wb') as file:
                    file.write('title={}\n'.format(title))
               #     file.write('date={}/{}\n'.format(datetime.datetime.strptime(description, '%Y%j').date().strftime('%Y-%m-%dT00:00:00Z'),
               #                                      datetime.datetime.strptime(description, '%Y%j').date().strftime('%Y-%m-%dT23:59:59Z')))
               #     file.write('geometry={}'.format(get_raster_wkt(daily_output_name)))

                ciop.log('INFO', 'Publish {}'.format(daily_output_name))


B01
0
{}


reporter:status:2019-10-31T19:43:05.954242 [INFO   ] [user process] key: B01
2019-10-31T19:43:05.954242 [INFO   ] [user process] key: B01
reporter:status:2019-10-31T19:43:05.954391 [INFO   ] [user process] Opening B01 - B01_2019045_2019150.tif
2019-10-31T19:43:05.954391 [INFO   ] [user process] Opening B01 - B01_2019045_2019150.tif


KeyError: 'band_is_interpolated'

In [58]:
results.keys()

['B01', 's', 'original_B01']

In [61]:
import datetime
#datetime.datetime.strptime(results['ndvi']['max_julian'], '%Y%j').date().strftime('%Y-%m-%dT23:59:59Z')
#datetime.date(2016, 8, 21)

In [68]:
for key in results.keys():
    
    if key == band_to_process: title = 'Sentinel-2 Smoothed NDVI'
    if key == 'original_{}'.format(band_to_process): title = 'Sentinel-2 Original NDVI'
    if key == 's_{}'.format(band_to_process): title = 'Sentinel-2 Smoothed NDVI s factor'
    
    with open(results[key]['file'].replace('.tif', '.properties'), 'wb') as file:
        file.write('title={0}\n'.format(title))
        file.write('date={0}\n'.format('/'.join([datetime.datetime.strptime(results[band_to_process]['min_julian'], '%Y%j').date().strftime('%Y-%m-%dT00:00:00Z'),
                                                datetime.datetime.strptime(results[band_to_process]['max_julian'], '%Y%j').date().strftime('%Y-%m-%dT23:59:59Z')])))
        file.write('geometry={0}'.format(get_raster_wkt(results[band_to_process]['file'])))

In [92]:
src_ds = gdal.Open('/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_0.pickle.tif')
ds = gdal.Open('ndvi.tif',  gdal.OF_UPDATE)

bands = dict()

for index in range(src_ds.RasterCount):

    ds.GetRasterBand(index+1).SetDescription(src_ds.GetRasterBand(index+1).GetDescription())

ds.FlushCache()

In [134]:
def get_raster_wkt(raster):
    src = gdal.Open(raster)
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)

    from osgeo import ogr
    from osgeo import osr

    # Setup the source projection - you can also import from epsg, proj4...
    source = osr.SpatialReference()
    source.ImportFromWkt(src.GetProjection())

    # The target projection
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)

    # Create the transform - this can be used repeatedly
    transform = osr.CoordinateTransformation(source, target)

    

    return box(transform.TransformPoint(ulx, lry)[0], 
       transform.TransformPoint(ulx, lry)[1],
       transform.TransformPoint(lrx, uly)[0],
       transform.TransformPoint(lrx, uly)[1]).wkt


In [87]:
src_ds.RasterCount

19

In [127]:
geoTransform = src.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * src.RasterXSize
miny = maxy + geoTransform[5] * src.RasterYSize
print [minx, miny, maxx, maxy]
data = None

[600000.0, 978060.0, 605490.0, 1000020.0]


In [90]:
bands

{'2019045': 0,
 '2019050': 1,
 '2019055': 2,
 '2019070': 3,
 '2019075': 4,
 '2019080': 5,
 '2019085': 6,
 '2019090': 7,
 '2019095': 8,
 '2019100': 9,
 '2019105': 10,
 '2019110': 11,
 '2019115': 12,
 '2019120': 13,
 '2019125': 14,
 '2019130': 15,
 '2019135': 16,
 '2019140': 17,
 '2019150': 18}

In [ ]:


ds.GetRasterBand(index + 1).SetDescription(dates.values[index])

        ds.FlushCache()

In [56]:
for file in (local_paths + [local_vrt]):
    
    print file

/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_0.pickle.tif
/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_1.pickle.tif
/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_2.pickle.tif
/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_3.pickle.tif
temp_vrt.xml


In [46]:



local_paths = [] 

for entry, row in df_references[df_references.output_type == 'ndvi'].iterrows():
    print row.output_type
    print row.enclosure
    local_paths.append(ciop.copy(row.enclosure, '.'))

ndvi
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000025-190909171543351-oozie-oozi-W/subtile/data/ndvi_tile_0_0.pickle.tif
ndvi
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000025-190909171543351-oozie-oozi-W/subtile/data/ndvi_tile_0_1.pickle.tif
ndvi
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000025-190909171543351-oozie-oozi-W/subtile/data/ndvi_tile_0_2.pickle.tif
ndvi
hdfs://sb-10-150-0-21.better.terradue.int:8020/ciop/run/ewf-wfp-02-03-01/0000025-190909171543351-oozie-oozi-W/subtile/data/ndvi_tile_0_3.pickle.tif


In [47]:
local_paths

['/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_0.pickle.tif',
 '/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_1.pickle.tif',
 '/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_2.pickle.tif',
 '/workspace/ewf-wfp-02-03-01/src/main/app-resources/merge/./ndvi_tile_0_3.pickle.tif']

In [49]:
import gdal

In [51]:
local_vrt = 'temp_vrt.xml'

ds = gdal.BuildVRT(local_vrt, local_paths)

ds.FlushCache()


In [53]:
translated_tif = 's_ndvi.tif'

In [54]:
gdal.Translate(translated_tif,
                       local_vrt)


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f8aac830c30> >

In [64]:
for index, row in df_references.iterrows():
    
   
    if index >=3:
        
        
        continue
        
    else:
        print index

0
1
2


In [120]:
title = 'S2A_MSIL2A_20190425T075611_N0211_R035_T36PXQ_20190425T112428.tif'

In [127]:
tile_id = title[38:44]
tile_id

'T36PXQ'